# Pré-Processamento dos Dados

## Título:

**Predição de preço de imóveis**

## Membros:

*   Adrisson Rogério Samersla
*   Nickolas Batista Mendonça Machado
*   Thayna Pires Baldão



# Setup

In [1]:
# Importando os pacotes necessários para a análise

import os

import pandas            as pd
import geopandas         as gpd
import numpy             as np
import scipy             as sc
import matplotlib.pyplot as plt
import seaborn           as sns

from google_drive_downloader import GoogleDriveDownloader as gdd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

import mapclassify

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.reset_option('max_colwidth')

In [2]:
# Baixando o dataset

dataset_dir = "../dataset"
has_dataset_dir = os.path.isdir(dataset_dir)
if (not has_dataset_dir):
  # Link of dataset folder: 
  # https://drive.google.com/file/d/1S4rBgtuogAGr_WIcF-FIPaULfGlB9MRs/view?usp=sharing
  gdd.download_file_from_google_drive(file_id='1S4rBgtuogAGr_WIcF-FIPaULfGlB9MRs',
                                      dest_path='../dataset.zip',
                                      showsize=True,
                                      unzip=True)

In [3]:
# Lendo a base de dados

df = pd.read_csv(dataset_dir + '/dataset.csv')
print("Formato dos dados: ", df.shape)
print("#Exemplos: {}".format(df.shape[0]))
print("#Atributos: {}".format(df.shape[1]))

Formato dos dados:  (1000000, 24)
#Exemplos: 1000000
#Atributos: 24


In [4]:
df.dtypes

id                           int64
property_id                 object
created_on                  object
operation                   object
property_type               object
place_name                  object
place_with_parent_names     object
country_name                object
state_name                  object
geonames_id                float64
lat_lon                     object
lat                        float64
lon                        float64
currency                    object
surface_total_in_m2        float64
surface_covered_in_m2      float64
floor                      float64
rooms                      float64
expenses                   float64
description                 object
title                       object
image_thumbnail             object
collected_on                object
price                      float64
dtype: object

In [5]:
df.head()

,id,property_id,created_on,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat_lon,lat,lon,currency,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description,title,image_thumbnail,collected_on,price
0,9641098,9e065715d8b6c69bc930c185901aaf71e3be3d54,2017-04-28,sell,apartment,Campo Belo,|Brasil|São Paulo|São Paulo|Campo Belo|,Brasil,São Paulo,NaN,"-23.616325,-46.67662",-23.616325,-46.676620,BRL,NaN,80.0,NaN,NaN,1065.0,Atualizado em 05/06/2017. O projeto arquitetôn...,"Apartamento com 2 quartos e 2 Suites, São Paul...",https://thumbs4.properati.com/2/tQXG8R8vA-PsM4...,2017-08,900000.0
1,7965368,f6b640522dd15dd3667850718f52aa02516b04ff,2017-05-25,sell,apartment,Porto Alegre,|Brasil|Rio Grande do Sul|Porto Alegre|,Brasil,Rio Grande do Sul,NaN,"-30.05175,-51.182068",-30.051750,-51.182068,BRL,91.0,91.0,NaN,NaN,NaN,"APARTAMENTO com 3 dormitórios , sendo 1 suíte,...",Apartamento em Petrópolis,https://thumbs4.properati.com/6/gy-6GDbU5Ccx_I...,2017-07,750000.0
2,6779668,39a954f5d56137f6fe547af707626eafa6a3869c,2015-06-26,sell,apartment,Bessa,|Brasil|Paraíba|João Pessoa|Bessa|,Brasil,Paraíba,NaN,NaN,NaN,NaN,BRL,NaN,NaN,NaN,NaN,NaN,Edifcio: 10(dez) apartamentos; 01(uma) vaga ga...,Venda Apartamento Joo Pessoa/Paraba: 3 quartos...,https://thumbs4.properati.com/8/DL8fMA8z1oRVgv...,2015-12,235000.0
3,10187959,ae0282c0e20210cfaf030e57edf85b9d5d6c13f4,2016-02-22,sell,apartment,São Lourenço,|Brasil|Paraná|Curitiba|São Lourenço|,Brasil,Paraná,NaN,"-25.413214,-49.271294",-25.413214,-49.271294,BRL,NaN,168.0,NaN,NaN,NaN,Linda cobertura no bairro São Lourenço com uma...,"Linda Cobertura duplex - 168 m² - 2 suítes, 2 ...",https://thumbs4.properati.com/2/J6USW1fALKsXov...,2016-06,880000.0
4,1147630,f83c204dd1baf29c6460826ee829be00f751f676,2016-09-09,sell,house,Vila Formosa,|Brasil|São Paulo|São Paulo|Vila Formosa|,Brasil,São Paulo,NaN,"-23.56229,-46.521248",-23.562290,-46.521248,BRL,NaN,145.0,NaN,2.0,NaN,"Lindo sobrado, muito bem construído com ótimo ...",Sobrado novo para venda na Vila Formosa.,https://thumbs4.properati.com/0/bXqmIc-TKt1wua...,2017-03,580000.0


# Seleção Manual de Atributos


In [6]:
ignore_columns = [
    'id',
    'property_id',
    'operation',
    'place_name',
    'place_with_parent_names',
    'country_name',
    'state_name',
    'geonames_id',
    'lat_lon',
    'currency',
    'floor',
    'description',
    'title',
    'image_thumbnail',
    'collected_on'
]

def manual_selection(dataset):
    return dataset.drop(columns=ignore_columns)

#filtered = manual_selection(df)
# for col in ignore_columns:
#     assert col not in manual_selection(df).columns

# Limpeza de dados

## Eliminando Inconsistências

- Moedas Estrangeiras

In [7]:
df.currency.astype('str').value_counts()

BRL    999806
MXN       103
USD        87
COP         4
Name: currency, dtype: int64

In [ ]:
def filter_currency(dataset):
    positions = dataset['currency'] == 'BRL'
    return dataset[positions]

#df = filter_currency(df).currency.astype('str').value_counts()

- Objetos Fora do Brasil

In [ ]:
df.state_name.astype('str').value_counts()

In [ ]:
def filter_latlon(dataset):
    data = dataset[['lat', 'lon']]

    # data = data.dropna()
    # points = gpd.points_from_xy(data.lon, data.lat, crs=map_df.crs)
    # points = gpd.GeoSeries(points)
    # positions_in = map_df.contains(points)

    positions = data.lat.notna().squeeze() & data.lon.notna().squeeze()

    lon_max = -30.0
    lon_min = -80.0
    lat_min = -40.0
    lat_max = 10.0
    return dataset.loc[
        positions & 
        (dataset.state_name != "Outros países") &
        ((data.lat > lat_min) & (data.lat < lat_max)) & 
        ((data.lon > lon_min) & (data.lon < lon_max))
    ]

#filter_latlon(df).state_name.astype('str').value_counts()

## Eliminando Outliers

In [ ]:
def remove_outliers(dataset, cols):
    Q1 = dataset[cols].quantile(0.25)
    Q3 = dataset[cols].quantile(0.75)
    IQR = Q3 - Q1

    return dataset[~((dataset[cols] < (Q1 - 1.5 * IQR)) |(dataset[cols] > (Q3 + 1.5 * IQR))).any(axis=1)]

# df = remove_outliers(df)
# print("Formato dos dados: ", df.shape)
# print("#Exemplos: {}".format(df.shape[0]))
# print("#Atributos: {}".format(df.shape[1]))

## Lidando com dados incompletos

- `expenses`

In [ ]:
def fill_expenses(dataset):
    dataset['expenses'].fillna(0.0, inplace=True)
    return dataset

# df = fill_expenses(df)
# df.isnull().sum()*100/1_000_000

- `surface_total_in_m2` e `surface_covered_in_m2`

In [ ]:
def fill_surface_total_with_surface_covered(dataset):
    dataset['surface_total_in_m2'].fillna(dataset['surface_covered_in_m2'], inplace=True)
    return dataset

def fill_surface_covered_with_surface_total(dataset):
    dataset['surface_covered_in_m2'].fillna(dataset['surface_total_in_m2'], inplace=True)
    return dataset

def fill_remaining_objects_without_surfaces(dataset):
    dataset['surface_total_in_m2'].fillna(dataset['surface_total_in_m2'].mean(), inplace=True)
    dataset['surface_covered_in_m2'].fillna(dataset['surface_covered_in_m2'].mean(), inplace=True)
    return dataset

def drop_remaining_objects_without_surfaces(dataset):
    dataset.dropna(subset=['surface_total_in_m2'], how='all', inplace=True)
    return dataset

# df = fill_surface_total_with_surface_covered(df)
# df = fill_surface_covered_with_surface_total(df)
# df = drop_remaining_objects_without_surfaces(df)
# df.isnull().sum()*100/1_000_000

- `rooms`

In [ ]:
def fill_rooms(dataset):
    dataset['surface_category'] = pd.qcut(dataset['surface_covered_in_m2'],4)
    dataset['rooms'] = dataset['rooms'].fillna(dataset.groupby('surface_category')['rooms'].transform('mean').round())
    return dataset.drop(columns=['surface_category'])
# df = fill_rooms(df)
# df.isnull().sum()*100/1_000_000

# Transformando dados

## Conversão simbólico-numérico

- `property_type`

In [ ]:
def convert_property_type(dataset):
    oe_style = OneHotEncoder()
    oe_results =  oe_style.fit_transform(dataset[['property_type']])
    dataset = dataset.join(pd.DataFrame(oe_results.toarray(), columns= oe_style.categories_))
    dataset.rename(columns=lambda s : str(s).replace(",", "").replace("(", "").replace(")", "").replace("\'", "") , inplace = True)
    dataset = dataset.drop(columns=['property_type'])
    return dataset[[c for c in dataset if c not in ['prices']] + ['prices']]

- `created_on`

In [ ]:
def convert_created_on(dataset):
    ord_enc = OrdinalEncoder()
    dataset['created_on'] = ord_enc.fit_transform(dataset[['created_on']])
    return dataset

# Normalização

## Por reescala

In [ ]:
def normalize_max_min(dataset, cols):
    for feature in cols:
        max_value = dataset[feature].max()
        min_value = dataset[feature].min()
        dataset[feature] = (dataset[feature] - min_value) / (max_value - min_value)
    return dataset

## Por padronização

In [ ]:
def standardizate(dataset, cols):
    for feature in cols:
        dataset[feature] = (dataset[feature] - dataset[feature].mean()) / (dataset[feature].std())
    return dataset

# Aplicando o Pré-Processamento

In [ ]:
print("#Exemplos: {}".format(df.shape[0]))
print("#Atributos: {}".format(df.shape[1]))
df = filter_currency(df)
df = filter_latlon(df)

df = manual_selection(df)
print("#Exemplos: {}".format(df.shape[0]))
print("#Atributos: {}".format(df.shape[1]))
df.dtypes

In [ ]:
df.isnull().sum()*100/df.shape[0]

In [ ]:
df = fill_expenses(df)
df = fill_surface_total_with_surface_covered(df)
df = fill_surface_covered_with_surface_total(df)
df = drop_remaining_objects_without_surfaces(df)
df.isnull().sum()*100/df.shape[0]

In [ ]:
# cols = ['surface_total_in_m2', 'surface_covered_in_m2', 'expenses']
# df = remove_outliers(df, cols)
# print("#Exemplos: {}".format(df.shape[0]))
# print("#Atributos: {}".format(df.shape[1]))
# df.isnull().sum()*100/df.shape[0]

In [ ]:
df = fill_rooms(df)
df.isnull().sum()*100/df.shape[0]

In [ ]:
cols = ['rooms']
df = remove_outliers(df, cols)
print("#Exemplos: {}".format(df.shape[0]))

In [ ]:
df = convert_property_type(df)
df = convert_created_on(df)
df.head()

In [ ]:
cols = ['created_on', 'lat', 'lon', 'rooms']
df = normalize_max_min(df, cols)

cols = ['surface_total_in_m2', 'surface_covered_in_m2', 'expenses']
df = standardizate(df, cols)
df.head()

In [ ]:
df.dtypes0
df.to_csv(dataset_dir + "/preprocessed.csv", index=False)